<a href="https://colab.research.google.com/github/duonghung86/Injury-severity-classification/blob/main/VCA_2_1_MLP_earlystopping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from psutil import virtual_memory,cpu_percent
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
print('Current system-wide CPU utilization %: ',cpu_percent())
#Remove all warning
import warnings
warnings.filterwarnings("ignore")

Your runtime has 202.3 gigabytes of available RAM

Current system-wide CPU utilization %:  0.0


In [12]:
# Basic packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import time
import os
# Preprocessing
from sklearn.preprocessing import StandardScaler # Standardization
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from collections import Counter
from imblearn.over_sampling import SMOTE, RandomOverSampler,BorderlineSMOTE
from imblearn.under_sampling import RandomUnderSampler,NearMiss,EditedNearestNeighbours
# Machine learning algos
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
# Metrics
from imblearn.metrics import geometric_mean_score
# Tensorflow
import tensorflow as tf
print(tf.__version__)
from tensorflow import feature_column  # for data wrangling
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow_addons.metrics import CohenKappa,F1Score
from tensorboard.plugins.hparams import api as hp

#Remove all warning
import warnings
warnings.filterwarnings("ignore")

2.1.0


In [3]:
url = 'https://github.com/duonghung86/Injury-severity-classification/blob/main/Prepared%20Texas%202019.zip?raw=true' 
data_path = tf.keras.utils.get_file(origin=url, fname=url.split('/')[-1].split('?')[0], extract=True)
data_path = data_path.replace('%20',' ').replace('.zip','.csv')

In [4]:
# Load data
df = pd.read_csv(data_path)
print(df.shape)
df.head(3)

(949856, 19)


,Prsn_Injry_Sev,Prsn_Age,Prsn_Gndr,Wthr_Cond,Light_Cond,Surf_Cond,Veh_Body_Styl,Prsn_Rest,Prsn_Drg_Rslt,Harm_Evnt,Rural,Crash_Speed_Limit,Road_Algn,Veh_Mod_Year,Weekend,Crash_season,Part_of_day,Collsn_type,Collsn_name
0,0,26,MALE,CLEAR,DAYLIGHT,DRY,"PASSENGER CAR, 2-DOOR",SHOULDER & LAP BELT,Not Applicable,MOTOR VEHICLE IN TRANSPORT,0,-1,"STRAIGHT, LEVEL",33,1,3,3,SAME DIRECTION,ONE STRAIGHT-ONE LEFT TURN
1,0,52,MALE,CLEAR,DAYLIGHT,DRY,"PASSENGER CAR, 2-DOOR",SHOULDER & LAP BELT,Not Applicable,MOTOR VEHICLE IN TRANSPORT,0,-1,"STRAIGHT, LEVEL",19,1,3,3,SAME DIRECTION,ONE STRAIGHT-ONE LEFT TURN
2,0,27,FEMALE,CLEAR,DAYLIGHT,DRY,PICKUP,SHOULDER & LAP BELT,Not Applicable,MOTOR VEHICLE IN TRANSPORT,1,-1,"CURVE, LEVEL",16,1,3,4,SAME DIRECTION,BOTH LEFT TURN


In [5]:
# Let's just use 80% of the total dataset
df, _ = train_test_split(df, test_size=0.95,stratify = df['Prsn_Injry_Sev'])
df.shape

(47492, 19)

In [6]:
y = df['Prsn_Injry_Sev']
print('All target values:')
print(y.value_counts())
X = df.drop(columns=['Prsn_Injry_Sev'])

All target values:
0    39627
1     5120
2     2262
3      398
4       85
Name: Prsn_Injry_Sev, dtype: int64


In [7]:
# %% Data wrangling -------------
# Classify variable type
emb_vars, ind_vars, num_vars = [], [], []
for var in X.columns:
    if X[var].dtypes == 'O':
        if len(X[var].unique()) > 5:
            emb_vars.append(var)
        else:
            ind_vars.append(var)
    else:
        num_vars.append(var)
print('Numerical variables are ', num_vars)
print('Categorical variables that have at most 5 categories are ', ind_vars)
print('Categorical variables that have more than 5 categories are ', emb_vars)

# Create feature columns
feature_columns = []
# numeric cols
for header in num_vars:
    feature_columns.append(feature_column.numeric_column(header))
# bucketized cols
# age = feature_column.numeric_column('Prsn_Age')
# age_buckets = feature_column.bucketized_column(age, boundaries=[16, 22, 35, 55, 65])
# feature_columns.append(age_buckets)
# indicator_columns
for col_name in ind_vars:
    categorical_column = feature_column.categorical_column_with_vocabulary_list(
        col_name, X[col_name].unique())
    indicator_column = feature_column.indicator_column(categorical_column)
    feature_columns.append(indicator_column)
# embedding columns
for col_name in emb_vars:
    emb_column = feature_column.categorical_column_with_vocabulary_list(
        col_name, X[col_name].unique())
    col_embedding = feature_column.embedding_column(emb_column, dimension=5)
    feature_columns.append(col_embedding)

# Convert all setup into new dataset
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
X = feature_layer(dict(X)).numpy()
print('New shape of the input data set:',X.shape)

Numerical variables are  ['Prsn_Age', 'Rural', 'Crash_Speed_Limit', 'Veh_Mod_Year', 'Weekend', 'Crash_season', 'Part_of_day']
Categorical variables that have at most 5 categories are  ['Prsn_Gndr', 'Prsn_Drg_Rslt', 'Collsn_type']
Categorical variables that have more than 5 categories are  ['Wthr_Cond', 'Light_Cond', 'Surf_Cond', 'Veh_Body_Styl', 'Prsn_Rest', 'Harm_Evnt', 'Road_Algn', 'Collsn_name']
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
New shape of the input data set: (47492, 59)


In [8]:
# %% Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=48)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=48)

print('Training features shape:', X_train.shape)
print('Validation features shape:', X_val.shape)
print('Test features shape:', X_test.shape)

# %% standardization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

Training features shape: (30394, 59)
Validation features shape: (7599, 59)
Test features shape: (9499, 59)


In [13]:
y_dict = Counter(y_train)
start = time.time()
res = RandomUnderSampler(sampling_strategy={0: y_dict[1]})
print('sampling #1 ...')
X_train, y_train = res.fit_resample(X_train, y_train)
res = BorderlineSMOTE(sampling_strategy='not majority')
print('sampling #2 ...')
X_train, y_train = res.fit_resample(X_train, y_train)
end = time.time()
res_time = end-start
Counter(y_train),res_time

sampling #1 ...
sampling #2 ...


(Counter({0: 3277, 1: 3277, 2: 3277, 3: 3277, 4: 3277}), 1.6237106323242188)

In [35]:
es = EarlyStopping(monitor='cohen_kappa',
                   verbose=1, patience=10, mode='max',
                   restore_best_weights=True)
def create_mlp():
    MLP = Sequential([Dense(10,activation='relu',
                           input_dim=X_train.shape[1]),
                      Dropout(0.5),
                      Dense(5, activation='softmax')])
    MLP.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[CohenKappa(num_classes=5,sparse_labels=True)])
    return MLP

In [36]:
model = create_mlp()
start = time.time()
model.fit(X_train, y_train,
                    callbacks=[es],
                 #   class_weight = cls_wgt,
                    validation_data=(X_val, y_val.values),
                    verbose=1, epochs=50)
end = time.time()
# use the model to make predictions with the test data
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
# get the evaluation metrics
get_accs(y_test.values, y_pred, end-start)

Train on 16385 samples, validate on 7599 samples
Epoch 1/50
16385/16385 [==============================] - 1s 53us/sample - loss: 1.5087 - cohen_kappa: 0.1825


InvalidArgumentError:  axis = 1 not in [-1, 1)
	 [[{{node metrics/cohen_kappa/StatefulPartitionedCall/confusion_matrix/stack_1}}]] [Op:__inference_distributed_function_461001]

Function call stack:
distributed_function


# ALL mini functions



In [14]:
# %% Function to compare the prediction and true labels
def get_accs(label, prediction, show=False):
    cm = confusion_matrix(label, prediction)
    length = cm.shape[0]
    num_cases = len(label)
    # global accuracy
    glb_acc = np.trace(cm) / len(label)
    ind_accs = cm / np.sum(cm, axis=1)[:, np.newaxis]
    accs = [ind_accs[i, i] for i in range(length)]
    index = ['Class {}'.format(i) for i in range(length)]
    # Global accuracy
    accs.append(glb_acc)
    index.append('Overall Accuracy')
    # G-mean
    accs.append(geometric_mean_score(label, prediction, correction=0.001))
    index.append('G-mean')
    # Average perf
    accs.append((glb_acc + accs[-1]) / 2)
    index.append('Avg_Pfm')
    if show:
        plt.figure(figsize=(14, 6))
        plt.subplot(1, 2, 1)
        sns.heatmap(cm, xticklabels=np.arange(length), yticklabels=np.arange(length),
                    annot=True, fmt='g', cmap="YlGnBu")
        plt.xlabel('Prediction')
        plt.ylabel('Label')
        plt.title('Confusion matrix')
        plt.subplot(1, 2, 2)
        sns.heatmap(ind_accs * 100, xticklabels=np.arange(length), yticklabels=np.arange(length),
                    annot=True, fmt='.2f', cmap="YlGnBu")
        plt.xlabel('Prediction')
        plt.ylabel('Label')
        plt.title('Normalized confusion matrix (%)')
        plt.show()
    out = np.array(accs).reshape(1, len(accs))
    return pd.DataFrame(out, columns=index)

def show_evolution(moni):
    hist = pd.DataFrame(monitor.history)
    no_metrics = np.int(hist.shape[1]/2)
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(6, 2), dpi=150)
    for i in range(2):
      hist.iloc[:,[i,no_metrics+i]].plot(ax=axes[i])
    plt.show()
# %% Produce an evaluation on the MLP model
def evaluation(model, monitor, time, name):
    # use the model to make predictions with the test data
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    # Show evolution of the training process
    # show_evolution(monitor)
    # get the evaluation metrics
    result = get_accs(y_test.values, y_pred)
    result['Training Time'] = np.round(time, 3)
    result.index = [name]
    return result

In [15]:
wgt=None
clfs = [LogisticRegression(solver = 'lbfgs',class_weight=wgt),
        DecisionTreeClassifier(class_weight=wgt),
        RandomForestClassifier(max_depth=4,class_weight=wgt)]
clf_names = ['LR','DT','RF']
rsts = pd.DataFrame()
for model, name in zip(clfs,clf_names):
    start = time.time()
    print(name)
    model.fit(X_train, y_train.values)
    # use the model to make predictions with the test data
    y_pred = model.predict(X_test)
    end= time.time()
    # get the evaluation metrics
    result = get_accs(y_test.values,y_pred)
    result['Training Time'] = np.round(end-start,3)
    result.index = [name]
    rsts = rsts.append(result)
print(rsts.iloc[:,-4:])

LR
DT
RF
    Overall Accuracy    G-mean   Avg_Pfm  Training Time
LR          0.498052  0.375033  0.436543          0.722
DT          0.422781  0.259401  0.341091          0.293
RF          0.503737  0.331783  0.417760          0.918


# MLP functions


In [16]:
def early_stops(metric_name):
    es = EarlyStopping(monitor='val_'+ metric_name,
                   verbose=1,
                   patience=10,
                   mode='max',
                   restore_best_weights=True)
    return es

# Ordinal multiclass


In [21]:
early_stop = {'accuracy':  tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy'),
              'cohen_kappa': CohenKappa(num_classes=5,
                                        #sparse_labels=True
                                       ),
              'f1_score': F1Score(num_classes=5,average="micro",threshold=0.5),
               }

In [18]:
def create_mlp(metric):
    MLP = Sequential([Dense(10,
                           activation='relu',
                           input_dim=X_train.shape[1]
                           ),
                      Dropout(0.5),
                      Dense(5, activation='softmax')])
    MLP.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                #loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=[metric]
               )
    return MLP

In [24]:
for name, metric in early_stop.items():
    print(name)
    model = create_mlp(metric)
    start = time.time()
    monitor = model.fit(X_train, y_train,
                    callbacks=[early_stops(name)],
                    validation_data=(X_val, y_val.values),
                    verbose=1, epochs=50)
    end = time.time()
    rsts = rsts.append(evaluation(model, monitor, end - start,  'MLP '+ name))
print(rsts.iloc[:,-4:])

accuracy
Train on 16385 samples, validate on 7599 samples
Epoch 1/50
16385/16385 [==============================] - 1s 76us/sample - loss: 1.5625 - accuracy: 0.3056 - val_loss: 1.4946 - val_accuracy: 0.4614
Epoch 2/50
16385/16385 [==============================] - 1s 43us/sample - loss: 1.4511 - accuracy: 0.4283 - val_loss: 1.4239 - val_accuracy: 0.5469
Epoch 3/50
16385/16385 [==============================] - 1s 43us/sample - loss: 1.4243 - accuracy: 0.4538 - val_loss: 1.3844 - val_accuracy: 0.5840
Epoch 4/50
16385/16385 [==============================] - 1s 43us/sample - loss: 1.4169 - accuracy: 0.4608 - val_loss: 1.4055 - val_accuracy: 0.5364
Epoch 5/50
16385/16385 [==============================] - 1s 44us/sample - loss: 1.4091 - accuracy: 0.4671 - val_loss: 1.4066 - val_accuracy: 0.5245
Epoch 6/50
16385/16385 [==============================] - 1s 43us/sample - loss: 1.4037 - accuracy: 0.4684 - val_loss: 1.4002 - val_accuracy: 0.5434
Epoch 7/50
16385/16385 [========================

InvalidArgumentError:  axis = 1 not in [-1, 1)
	 [[{{node metrics/cohen_kappa/StatefulPartitionedCall/confusion_matrix/stack_1}}]] [Op:__inference_distributed_function_230791]

Function call stack:
distributed_function


# One-hot encoded multiclass


In [29]:
 early_stop = {'auc':       tf.keras.metrics.AUC(name='auc'),
     'accuracy':  tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
               'precision': tf.keras.metrics.Precision(name='precision'),
               'recall':    tf.keras.metrics.Recall(name='recall'),
               
               }

In [30]:
def create_mlp(metric):
    MLP = Sequential([Dense(10,
                           activation='relu',
                           input_dim=X_train.shape[1]
                           ),
                      Dropout(0.5),
                      Dense(5, activation='softmax')])
    MLP.compile(optimizer='adam',
                #loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=[metric]
               )
    return MLP

In [31]:
#rsts = rsts.iloc[:6,:].copy()

In [31]:
for name, metric in early_stop.items():
    print(name)
    model = create_mlp(metric)
    start = time.time()
    monitor = model.fit(X_train, pd.get_dummies(y_train).values,
                    callbacks=[early_stops(name)],
                    validation_data=(X_val, pd.get_dummies(y_val).values),
                    verbose=0, epochs=50)
    end = time.time()
    rsts = rsts.append(evaluation(model, monitor, end - start,  'MLP 1H '+ name))
print(rsts.iloc[:,-4:])

auc
Restoring model weights from the end of the best epoch.
Epoch 00022: early stopping
accuracy
Restoring model weights from the end of the best epoch.
Epoch 00027: early stopping
precision
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
recall
Restoring model weights from the end of the best epoch.
Epoch 00025: early stopping
                  Overall Accuracy    G-mean   Avg_Pfm  Training Time
LR                        0.498052  0.375033  0.436543          0.722
DT                        0.422781  0.259401  0.341091          0.293
RF                        0.503737  0.331783  0.417760          0.918
MLP accuracy              0.508159  0.370940  0.439550         15.952
MLP accuracy              0.469734  0.359716  0.414725         13.669
MLP accuracy              0.567639  0.343664  0.455651          8.656
MLP accuracy              0.509843  0.385545  0.447694         20.709
MLP accuracy              0.593852  0.362608  0.478230         10.016
MLP 

In [33]:
rsts.to_csv('VCA_Res_Early_stopping.csv')